### Crop and Save Images

In [ ]:
import os
import cv2
import numpy as np
from typing import List
from tqdm import tqdm
from ipynb.fs.full.FinalProject import detect_circles, crop_to_circle



def save_to_file(imgs: List[np.ndarray], label: str, num: int):
    if not os.path.exists(f"data/processed/{label}"):
            os.makedirs(f"data/processed/{label}")
            
    for i, img in enumerate(imgs):
        if img is None or min(img.shape) < 48:
            continue
        
        path = os.path.join("data", "processed", label, f"image{i}-{num}.jpg")
        cv2.imwrite(path, img)


images_with_labels = list(enumerate(zip(train_images, train_labels)))
for i, (img, label) in tqdm(images_with_labels):
    circles = detect_circles(img, max=5)
    cis = crop_to_circle(img, circles)
    save_to_file(cis, label, i)

### Swap Numbers

In [ ]:
import os
import re

def rename_files(folder: str):
    # Regular expression to match the file names
    pattern = re.compile(r'image(\d+)-(\d+)\.jpg')
    
    for filename in os.listdir(folder):
        match = pattern.match(filename)
        if match:
            x, y = match.groups()
            new_filename = f'image{y}-{x}.jpg'
            old_path = os.path.join(folder, filename)
            new_path = os.path.join(folder, new_filename)
            os.rename(old_path, new_path)
            print(f'Renamed: {filename} -> {new_filename}')

# Example usage
rename_files('data/ball/billiard_ball')

### Remove Ball Images

In [ ]:
import os

def remove_matching_files(folder1: str, folder2: str):
    subfolders = os.listdir(folder2)
    
    for subfolder in subfolders:
        path1 = os.path.join(folder1, subfolder)
        path2 = os.path.join(folder2, subfolder)
        
        if os.path.exists(path1) and os.path.exists(path2):
            files1 = set(os.listdir(path1))
            files2 = set(os.listdir(path2))
            
            matching_files = files1.intersection(files2)
            
            for file in matching_files:
                file_path = os.path.join(path2, file)
                os.remove(file_path)
                print(f'Removed: {file_path}')

# Example usage
remove_matching_files('data/ball', 'data/processed')